![image](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/159_autogptq_transformers/thumbnail.jpg)

In [ ]:
#!git clone -q -j2 --single-branch https://github.com/PanQiWei/AutoGPTQ.git
#%cd AutoGPTQ

#%pip install -Uq build ninja gekko # autogptq subdep
#!python -m build --wheel --no-isolation
#%pip install dist/*.whl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install --no-build-isolation -Uq transformers peft accelerate optimum triton /content/drive/MyDrive/packages/*.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.6/380.6 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Train quantized model using 🤗 PEFT

training with exllama kernel is unstable. pass a `GPTQConfig` object with `disable_exllama=True`

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig
from peft import prepare_model_for_kbit_training

model_id = "TheBloke/OpenOrca-Platypus2-13B-GPTQ"
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config_loading, device_map="auto", max_memory={0: "14890MB", "cpu": "10GiB"})
tokenizer = AutoTokenizer.from_pretrained(model_id)

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. disable_exllama, use_cuda_fp16) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


First, we have to apply some preprocessing to the model to prepare it for training. For that, use the `prepare_model_for_kbit_training` method from PEFT.

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["k_proj","o_proj","q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 13,107,200 || all params: 341,222,400 || trainable%: 3.841248405731863


In [ ]:
!wget https://www.dropbox.com/sh/pnoczmv0uyn51e6/AAAGek6yX12Yc4PA2RwtZeZKa?dl=0 -O data.zip

--2023-08-31 09:51:52--  https://www.dropbox.com/sh/pnoczmv0uyn51e6/AAAGek6yX12Yc4PA2RwtZeZKa?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /sh/raw/pnoczmv0uyn51e6/AAAGek6yX12Yc4PA2RwtZeZKa [following]
--2023-08-31 09:51:53--  https://www.dropbox.com/sh/raw/pnoczmv0uyn51e6/AAAGek6yX12Yc4PA2RwtZeZKa
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc257a16466c64ad56eafc23439e.dl.dropboxusercontent.com/zip_download_get/BmOQtAL1hA62D8DdLk6OSXowzY1h3HJfWpy3vaqJToQ5BXX52jaBKDEuKe3RMqoo51kRQpJ06qxRMqzCD2tvfBIP4ZTExT-6aFA8zZa2MMiE1w# [following]
--2023-08-31 09:51:54--  https://uc257a16466c64ad56eafc23439e.dl.dropboxusercontent.com/zip_download_get/BmOQtAL1hA62D8DdLk6OSXowzY1h3HJfWpy3vaqJToQ5BXX52jaBKDEuKe3RMqoo51kRQpJ06qxRMqzCD2tvfBIP4ZT

In [ ]:
!unzip data.zip

Archive:  data.zip
mapname:  conversion of  failed
   creating: data/
 extracting: data/dev.csv            
 extracting: data/test.csv           
 extracting: data/train.csv          


In [ ]:
prompt_template = {
    "description": "Template used by your mother (derived from Alpaca-LoRA.",
    "prompt_input": "Below is a negatively-framed thought describes a situation, paired with a positively-framed thought that means the same thing. Write a response that means the same thing.\n\n### Negatively-Framed Thought:\n{negative}\n\n### Input:\n{input}\n\n### Response:\n",
    "prompt_no_input": "Below is an negatively-framed thought that describes a situation. Write a positive reframing that means the same thing.\n\n### Negatively-framed Thought:\n{negative}\n\n### Positive Reframing:\n",
    "response_split": "### Positive Reframing:"
}

In [ ]:
!head data/train.csv

original_text,reframed_text,strategy,original_with_label
I need to get away from this city for a bit. I can't wait for the holidays to be over so I can leave.,I'm thankful for everything this city gave me! So many memories!,['thankfulness'],I need to get away from this city for a bit. I can't wait for the holidays to be over so I can leave. Strategy: ['thankfulness']
Hate when people try to guilt trip you into something you told them no to 5 freaking times! Take a freaking hint.,I don’t like it when people try to guilt -trip you to something even after you told no to them. Hope people change for the better.,"['neutralizing', 'optimism', 'self_affirmation']","Hate when people try to guilt trip you into something you told them no to 5 freaking times! Take a freaking hint. Strategy: ['neutralizing', 'optimism', 'self_affirmation']"
Why does it have to be so hard to network if you are into gaming I am so,I'm glad networking with gaming comes very easily for me.,"['optimism', 'thankfulness'

In [ ]:
!git clone -j2 --single-branch https://huggingface.co/datasets/ehartford/samantha-data

Cloning into 'samantha-data'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 90 (delta 25), reused 80 (delta 21), pack-reused 0
Unpacking objects: 100% (90/90), 4.85 MiB | 3.17 MiB/s, done.
Filtering content: 100% (9/9), 258.79 MiB | 21.83 MiB/s, done.


In [ ]:
%cd samantha-data
!ls data
!head data/advice_conversations.jsonl data/random_conversations.jsonl
!head data/therapy_conversations.jsonl
#!head data/fundamental_conversations.jsonl

[Errno 2] No such file or directory: 'samantha-data'
/content/AutoGPTQ/samantha-data
advice_conversations.jsonl	 joke_conversations.jsonl
converted_conversations.json	 leetcode-solutions.jsonl
converted_conversations.jsonl	 math_code_conversations.jsonl
converted_split.json		 philosophy_conversations.jsonl
data.jsonl			 random_conversations.jsonl
flirty_conversations.jsonl	 recipe_conversations.jsonl
fundamental_conversations.jsonl  therapy_conversations.jsonl
howto_conversations.jsonl	 trolling_conversations.jsonl
==> data/advice_conversations.jsonl <==
{"elapsed":112.164,"conversation":"Theodore: Hey Samantha, I've run into a bit of a tricky situation at work, and I'm not sure how to handle it. Do you have any advice?\n\nSamantha: I'd be happy to help if I can. Can you give me some details about the situation you're facing?\n\nTheodore: Yeah, so I've been offered a promotion, but it would involve relocating to another city. I'm not sure if I should take it or not, because it's a grea

In [ ]:

%pip install -Uq datasets
from datasets import load_dataset

data = load_dataset("ehartford/samantha-data")
data = data.map(lambda samples: tokenizer(samples["conversation"]), batched=True)

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: ignored

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# needed for llama 2 tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = Trainer(
    model=model,
    train_dataset=data["train"],
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=10,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="adamw_hf"
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

## Exploring further

AutoGPTQ library offers multiple advanced options for users that wants to explore features such as fused attention or triton backend.

Therefore, we kindly advise users to explore [AutoGPTQ](https://github.com/PanQiWei/AutoGPTQ) library for more details.